In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from skimage.io import imshow
from datetime import datetime
import math
from PIL import Image

import vgg16
import utils

In [2]:
batch_size = 64
image_num = 1664 #The number of image data
batch_num = math.ceil(image_num/batch_size)
sample_size = 3
epoch_num = 1000
classes_num = 5
MOVING_AVERAGE_DECAY = 0.9999

ckpt_loc = 'TL_ckpt' 
eval_loc = 'eval_dir'
data_loc = 'hanseul'

dictionary = {'사람':0,'고양이':1,'강아지':2,'차':3,'배':4}

In [3]:
if not os.path.isdir(ckpt_loc):
    os.mkdir(ckpt_loc)
if not os.path.isdir(eval_loc):
    os.mkdir(eval_loc)

In [4]:
with open('hanseul_data.txt','r') as f:
    lines = f.readlines() #line[:-3] : image_name line[-2:-1] : image_label
    
image = np.ndarray([image_num,224,224,3])
label = np.ndarray([image_num])

i = 0
for line in lines:
    if i == image_num: break
    label[i] = np.asanyarray(int(line[-2:-1])).reshape([1])
    image[i] = np.asanyarray(Image.open(line[:-3])).reshape([1,224,224,3])
    i += 1

In [ ]:
with tf.Graph().as_default() as g:
    with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.6)))) as sess:
        #For saving checkpoint

        ckpt = tf.train.get_checkpoint_state(ckpt_loc)

        print('program is started')
        images = tf.placeholder("float32", [None, 224, 224, 3],name = 'images')
        labels = tf.placeholder("int64",[None], name = 'labels')

        figure = plt.figure(figsize=(20,sample_size))
        vgg = vgg16.Vgg16(fig = figure)
 
        with tf.name_scope("content_vgg"):
            vgg.build(images,labels)
            sess.run(tf.global_variables_initializer())
            
            filter_img = tf.summary.FileWriter('filter',sess.graph)
            saver = tf.train.Saver()
            
            start = datetime.now()
            print(' Running starts!')
            
            for epoch in range(epoch_num):
                img_count=0
                loss_sum = 0.0
                acc_sum = 0.0 
                
                for batch in range(batch_num):
                    feed_dict = {images: image[img_count:img_count+batch_size], labels: label[img_count:img_count+batch_size]}

                    loss, train, acc = sess.run([vgg.cost, vgg.train, vgg.accuracy], feed_dict=feed_dict)

                    img_count += batch_size
                    global_step = img_count + image_num*(epoch+1)
                    
                    loss_sum +=  loss
                    acc_sum += acc
                    
                loss = np.sum(loss_sum)/batch_num
                acc = np.sum(acc_sum)/batch_num
                
                print('[',datetime.now(),"%2d" %(epoch+1),"epoch] loss : %7.4f" % loss, "accuracy : %.6f" % acc)
                
                if epoch % 10 == 0:
                    saver.save(sess,os.path.join(ckpt_loc,"TL-model"),global_step = global_step) #write_meta_graph=False meta 원하지 않음

            print(datetime.now(),' Running finished!')
            print('-img.close')
            filter_img.close()


program is started
C:\Users\koo\jupyterNotebook\Transfer Learning\vgg16.npy
npy file loaded
build model started
build model finished: 3s
 Running starts!
[ 2018-07-25 18:03:06.917768  1 epoch] loss : 122985.4519 accuracy : 0.819111
[ 2018-07-25 18:03:31.507175  2 epoch] loss : 3904.9838 accuracy : 0.984375
[ 2018-07-25 18:03:46.462030  3 epoch] loss : 1188.6663 accuracy : 0.996394
[ 2018-07-25 18:04:01.415885  4 epoch] loss : 75.8933 accuracy : 0.999399
[ 2018-07-25 18:04:16.368741  5 epoch] loss :  0.0000 accuracy : 1.000000
[ 2018-07-25 18:04:31.330596  6 epoch] loss :  0.0000 accuracy : 1.000000
[ 2018-07-25 18:04:46.284452  7 epoch] loss :  0.0000 accuracy : 1.000000
[ 2018-07-25 18:05:01.261308  8 epoch] loss :  0.0000 accuracy : 1.000000
[ 2018-07-25 18:05:16.217164  9 epoch] loss :  0.0000 accuracy : 1.000000
[ 2018-07-25 18:05:31.200021 10 epoch] loss :  0.0000 accuracy : 1.000000
